Ce notebook a pour but d'explorer différentes méthodes pour utiliser et fine-tuner des réseaux préentrainés

In [1]:
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [10]:
#on load un dataset d'image depuis pytorch
dataset = dset.CIFAR10(root='data',
                          download=True,
                            transform=transforms.Compose([
                                transforms.Resize(64),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                            ]))

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data


In [ ]:
#on affiche une image du dataset avec matplotlib
lenD = len(dataset)
print(lenD)
i = random.randint(0, lenD)
img = dataset[i][0]
Vimg = img.numpy().transpose(1,2,0)
plt.imshow(Vimg)

## I/ Utiliser des réseaux fournis avec Pytorch

In [59]:
from torchvision.models import resnet50, ResNet50_Weights

# Best available weights (currently alias for IMAGENET1K_V2)
# Note that these weights may change across versions
my_model = resnet50(weights=ResNet50_Weights.DEFAULT)
print(type(my_model))

# # Old weights with accuracy 76.130%
# resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
# # New weights with accuracy 80.858%
# resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
# # Strings are also supported
# resnet50(weights="IMAGENET1K_V2")
# # No weights - random initialization
# resnet50(weights=None)

<class 'torchvision.models.resnet.ResNet'>


... il est important de preprocesser de la bonne manière en focntion du modèle utiliser : en pytorch c'est simple il suffit de récupérer la fonction transforms() qui doit être présent comme attribut du "weight" choisi pour le modèle.

In [58]:
# Initialize the Weight Transforms
weights = ResNet50_Weights.DEFAULT
preprocess = weights.transforms()


# Apply it to the input image
img_transformed = preprocess(img)

In [ ]:
#affichage de l'image transformée et de l'image dans le meme cadre 
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].set_title('Original')
ax[0].imshow(Vimg)
ax[1].set_title('Transformed')
ax[1].imshow(img_transformed.numpy().transpose(1,2,0))


...ne pas oublier que le modele charger peut avoir un comportement différent en train et eval

In [ ]:
# Set model to eval mode
my_model.eval()
#my_model.train()

In [ ]:
# # List available models
# all_models = list_models()
# classification_models = list_models(module=torchvision.models)

# # Initialize models
# m1 = get_model("mobilenet_v3_large", weights=None)
# m2 = get_model("quantized_mobilenet_v3_large", weights="DEFAULT")

# # Fetch weights
# weights = get_weight("MobileNet_V3_Large_QuantizedWeights.DEFAULT")
# assert weights == MobileNet_V3_Large_QuantizedWeights.DEFAULT

# weights_enum = get_model_weights("quantized_mobilenet_v3_large")
# assert weights_enum == MobileNet_V3_Large_QuantizedWeights

# weights_enum2 = get_model_weights(torchvision.models.quantization.mobilenet_v3_large)
# assert weights_enum == weights_enum2

In [ ]:
import torchviz
#on visualise le modèle avec torchviz
x = torch.randn(1, 3, 224, 224)
y = my_model(x)
y.detach_()
#on trnasforme en numpy
y = y.numpy()
y.reshape((20,50))

#on affiche l'image avec matplotlib
#plt.imshow(y)
# g = torchviz.make_dot(y.mean(), params=dict(my_model.named_parameters()))
# g.view()


## II/ Utiliser du code